### Counter with Message Passing (Go)

Consider the following flawed program in Go. It concurrently increments and decrements integer `x`, a global variable, `N` times.

In [82]:
%%writefile incdecrace.go
package main

const N = 20

var x int
var done chan bool

func incN() {
    for i := 0; i < N; i++ {x += 1}
    done <- true
}
func decN() {
    for i := 0; i < N; i++ {x -= 1}
    done <- true
}
func main() {
    x, done = 0, make(chan bool)
    go incN(); go decN()
    <- done; <- done;
    println("computed:", x)
    println("expected: 0")
}

Overwriting incdecrace.go


In [80]:
!go run incdecrace.go

computed: 0
expected: 0


For small values of `N`, it appears to run correctly. What is approximately the smallest value of `N` for which it does not run correctly? Give a brief explanation of why it runs correctly for small values! [2 points]

the program starts to work incorrectly for N at 5000. sometimes the program will work with N at 5000 but is inconsistent and will sometimes not work. This works for small values of N due to how go scheduler works. For small values Go is able to execute the routines in a fair interleaved manner however as we increase the value of N we also increase the chance we run into race conditions and both routines may interfere with each other updating x in a non fair way. 

Go has built-in race detection that can be enabled with `-race`. Races are detected at run-time, not at compile-time. Running the above program with a small value of `N` detects the race condition:

In [83]:
!go run -race incdecrace.go

Read at 0x000000554c00 by goroutine 7:
  main.decN()
      /home/ahmem73/Assignment10/incdecrace.go:13 +0x32

Previous write at 0x000000554c00 by goroutine 6:
  main.incN()
      /home/ahmem73/Assignment10/incdecrace.go:9 +0x4a

Goroutine 7 (running) created at:
  main.main()
      /home/ahmem73/Assignment10/incdecrace.go:18 +0x98

Goroutine 6 (finished) created at:
  main.main()
      /home/ahmem73/Assignment10/incdecrace.go:18 +0x8c
computed: 0
expected: 0
Found 1 data race(s)
exit status 66


The task is to fix the above program by making `x` local to a new process (goroutine), `counter`. Variable `x` is incremented and decremented by receiving messages from channels `inc` and `dec`. Process `counter` can also send `x` over a channel `val` as in the template below. [6 points]

In [6]:
%%writefile incdec.go
package main

const N = 1000000

var inc, dec, done chan bool
var val chan int

func counter(){
    var x int
    for {
        select{
            case <-inc:
                x++
            case <-dec:
                x--
            case val <- x:
                return
        }
    }
}


func incN() {
    for i := 0; i < N; i++ {
        inc <- true
    }
    done <- true
}

func decN() {
    for i := 0; i < N; i++ {
       dec <- true
    }
    done <- true
}
func main() {
    done = make(chan bool)
    inc, dec, val = make(chan bool), make(chan bool), make(chan int)
    go counter()
    go incN(); go decN()
    <- done; <- done;
    println("computed:", <- val)
    println("expected: 0")
}

Overwriting incdec.go


In [7]:
!go run -race incdec.go

computed: 0
expected: 0
